In [1]:
import yfinance as yf

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

import plotly.express as px
import joblib

In [2]:
df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
df_nvidia_to_csv = yf.download("NVDA", start="2020-01-01", end="2025-08-13")


apple_path = 'datasets/apple.csv'
microsoft_path = 'datasets/microsoft.csv'
amazon_path = 'datasets/amazon.csv'
nvidia_path = 'datasets/nvidia.csv'

df_apple_to_csv.to_csv(apple_path)
df_microsoft_to_csv.to_csv(microsoft_path)
df_amazon_to_csv.to_csv(amazon_path)
df_nvidia_to_csv.to_csv(nvidia_path)

df_apple = pd.read_csv(apple_path)
df_microsoft = pd.read_csv(microsoft_path)
df_amazon = pd.read_csv(amazon_path)
df_nvidia = pd.read_csv(nvidia_path)

C:\Users\acer\AppData\Local\Temp\ipykernel_23468\126190450.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_apple_to_csv = yf.download("AAPL", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_23468\126190450.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_microsoft_to_csv = yf.download("MSFT", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_23468\126190450.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_amazon_to_csv = yf.download("AMZN", start="2020-01-01", end="2025-08-13")
[*********************100%***********************]  1 of 1 completed
C:\Users\acer\AppData\Local\Temp\ipykernel_23468\126190450.py:4: FutureWarning: YF.download() has changed argument auto_adjus

In [3]:
df_apple['Date'] = df_apple['Price']
df_apple = df_apple.drop(index=[0,1], columns='Price')

df_microsoft['Date'] = df_microsoft['Price']
df_microsoft = df_microsoft.drop(index=[0,1], columns='Price')

df_amazon['Date'] = df_amazon['Price']
df_amazon = df_amazon.drop(index=[0,1], columns='Price')

df_nvidia['Date'] = df_nvidia['Price']
df_nvidia = df_nvidia.drop(index=[0,1], columns='Price')

In [4]:
#INITIAL
dfs = [df_apple, df_amazon, df_microsoft, df_nvidia]
window = 14
short_period = 12
long_period = 26
signal_line_period = 9
years = list(range(2020,2026))
stocks = ['AAPL', 'AMZN', 'MSFT', 'NVDA']
X_train, X_test, y_train, y_test = None, None, None, None

In [5]:
#CONVERTING TO CORRECT FEATURE TYPES
def convert_type(df):
    df['Close'] = df['Close'].astype(float)
    df['High'] = df['High'].astype(float)
    df['Low'] = df['Low'].astype(float)
    df['Open'] = df['Open'].astype(float)
    df['Volume'] = df['Volume'].astype(int)
    df['Date'] = pd.to_datetime(df['Date'])

In [6]:
#FEATURE ENGINEERING
def engineer_features(df):
    df['Price Change'] = df['Close'].diff().fillna(0)

    df['Gain'] = df.loc[df['Price Change'] > 0, 'Price Change']
    df['Loss'] = -df.loc[df['Price Change'] < 0, 'Price Change']

    df['Gain'] = df['Gain'].fillna(0)
    df['Loss'] = df['Loss'].fillna(0)

    df['Average Gain'] = df['Gain'].rolling(window=window).mean().fillna(0)
    df['Average Loss'] = df['Loss'].rolling(window=window).mean().fillna(0)

    df['RS'] = df['Average Gain'] / df['Average Loss']

    df['RSI'] = 100 - (100 / (1 + df['RS']))

    df['Short EMA'] = df['Close'].ewm(span=short_period,adjust=False).mean()
    df['Long EMA'] = df['Close'].ewm(span=long_period, adjust=False).mean()

    df['MACD'] = df['Short EMA'] - df['Long EMA']

    df['Signal Line'] = df['MACD'].ewm(span=signal_line_period, adjust=False).mean()
    
    df['MACD Histogram'] = df['MACD'] - df['Signal Line']
    
    return df

In [7]:
#DROPPING NULL AND UNIMPORTANT COLUMNS/ROWS
def drop_features_and_na(df):
    df = df.drop(columns=['Gain', 'Loss'])
    df = df.dropna()
    return df

In [8]:
#MODEL TRAINING AND EVALUATION
def split_data(df):
    X = df.drop(columns=['Date', 'Close'])
    y = df['Close']

    return train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def create_pipeline():
    model = clone(Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LinearRegression())
        ]))
    return model

In [10]:
def evaluate(model):
    kfold = KFold(n_splits=6, random_state=30, shuffle=True)

    params = {'lr__n_jobs': [10,20,50]}
    cv = clone(GridSearchCV(estimator=model,param_grid=params,cv=kfold))

    cv.fit(X_train, y_train)
    
    return {"best_params": cv.best_params_, "best_score": cv.best_score_}, cv

In [11]:
#MAIN OUTPUT
models = []

for i,df in enumerate(dfs):
    X_train, X_test, y_train, y_test = None, None, None, None
    
    convert_type(df)
    df = engineer_features(df)
    df = drop_features_and_na(df)
    
    X_train, X_test, y_train, y_test = split_data(df)
    
    model = create_pipeline()
    
    evaluation, cv = evaluate(model)
    df = df.drop(columns='Close')
    
    print("Best parameter:", evaluation['best_params']['lr__n_jobs'])
    print("Best score:", evaluation['best_score'])
    
    models.append({'model': cv, 'X_test': X_test,'y_test': y_test})
    
    print(f"{stocks[i]}:")
    print(df.head())
    print()

Best parameter: 10
Best score: 0.9996741839719219
AAPL:
         High        Low       Open     Volume       Date  Price Change  \
15  77.281835  76.634581  76.941301  101832400 2020-01-22      0.272903   
16  77.177978  76.233660  76.781900  104472000 2020-01-23      0.369507   
17  78.088470  76.685278  77.344612  146537600 2020-01-24     -0.222206   
18  75.296578  73.632556  74.883593  161940000 2020-01-27     -2.260559   
19  76.897791  75.397996  75.497017  162234000 2020-01-28      2.110802   

    Average Gain  Average Loss        RS        RSI  Short EMA   Long EMA  \
15      0.508211      0.208906  2.432720  70.868581  75.349473  74.270927   
16      0.534604      0.208906  2.559060  71.902696  75.618521  74.480360   
17      0.534604      0.174406  3.065290  75.401506  75.811991  74.657821   
18      0.493720      0.335874  1.469955  59.513424  75.627918  74.654687   
19      0.644491      0.311551  2.068653  67.412415  75.796902  74.808141   

        MACD  Signal Line  MAC

In [12]:
for i in range(len(models)):
    model_ = models[i]['model']
    X_test_ = models[i]['X_test']
    y_test_ = models[i]['y_test']
    
    y_pred_ = model_.predict(X_test_)
    
    print("MSE:", mean_squared_error(y_true=y_test_, y_pred=y_pred_))
    print("R2 Score:", r2_score(y_true=y_test_, y_pred=y_pred_))

    print(f"Predicted values for {stocks[i]}:")
    print(y_pred_[0:10])

    print(f"Correct values for {stocks[i]}:")
    print(y_test_[0:10])
    
    joblib.dump(model,f"models/{stocks[i]}.pkl")
    
    if joblib.load(f"models/{stocks[i]}.pkl"):
        print(f"{stocks[i]} model saved!")

MSE: 0.6750612776150048
R2 Score: 0.9996383431827037
Predicted values for AAPL:
[133.82678573 226.0804412  229.18568132 190.12129958 173.15525241
 151.81310472 141.84407622 213.75729524 206.77115337 192.08404156]
Correct values for AAPL:
378     133.879730
1280    227.203445
1234    229.056870
994     191.558212
493     172.296021
687     151.331177
708     141.161713
1407    213.008255
1354    206.625504
1003    191.974686
Name: Close, dtype: float64
AAPL model saved!
MSE: 0.7486115541830004
R2 Score: 0.9994616581084482
Predicted values for AMZN:
[172.71282816 236.81055046 198.19159297 144.54340256 169.74176937
 119.684531   115.99934496 221.57529879 204.58754382 153.2784138 ]
Correct values for AMZN:
378     172.007996
1280    237.419998
1234    197.119995
994     145.889999
493     169.567505
687     118.540001
708     115.250000
1407    222.309998
1354    204.070007
1003    153.419998
Name: Close, dtype: float64
AMZN model saved!
MSE: 1.9466468837956903
R2 Score: 0.9997781495360968